In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Project/LoL/teamGold_2021-02-20.csv')
df

,gameId,platformId,gameCreation,gameDuration,winTeam,B00,B01,B02,B03,B04,B05,B06,B07,B08,B09,B10,B11,B12,B13,B14,B15,B16,B17,B18,B19,B20,B21,B22,B23,B24,B25,B26,B27,B28,B29,B30,B31,B32,B33,B34,...,R20,R21,R22,R23,R24,R25,R26,R27,R28,R29,R30,R31,R32,R33,R34,R35,R36,R37,R38,R39,R40,R41,R42,R43,R44,R45,R46,R47,R48,R49,R50,R51,R52,R53,R54,R55,R56,R57,R58,R59
0,2696218520,EUN1,1610107079251,1953,100,2500,2520,2919,4522,6395.0,8868.0,10704.0,13602.0,15652.0,17816.0,20701.0,22658.0,24849.0,28407.0,31183.0,33675.0,35888.0,37751.0,39777.0,43828.0,45970.0,46988.0,48428.0,50116.0,52652.0,54646.0,56111.0,57298.0,58848.0,60119.0,61133.0,62274.0,64644.0,66245.0,NaN,...,36384.0,40443.0,43826.0,45452.0,46463.0,48740.0,51060.0,53631.0,54845.0,58586.0,60128.0,64242.0,65630.0,66812.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5019660091,EUW1,1610083170348,1393,200,2500,2500,2959,4263,5776.0,7729.0,8929.0,10173.0,12205.0,13839.0,15782.0,19075.0,21391.0,23899.0,25540.0,26843.0,28961.0,31412.0,32550.0,34332.0,35751.0,37841.0,39544.0,41668.0,42853.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,45351.0,49019.0,50864.0,55436.0,55629.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4903548900,KR,1610034019092,1590,100,2500,2500,3013,5542,6525.0,8674.0,9894.0,11700.0,14722.0,16613.0,19115.0,21886.0,25497.0,28222.0,31479.0,33698.0,35321.0,36982.0,40184.0,42621.0,44408.0,49769.0,52497.0,54909.0,57476.0,59338.0,63252.0,64650.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38505.0,39471.0,41357.0,43151.0,46692.0,49419.0,50523.0,51308.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3736930681,NA1,1610198529096,1329,100,2500,2500,2900,4306,5881.0,7274.0,9769.0,11411.0,12950.0,14976.0,16618.0,18858.0,22576.0,25312.0,27010.0,29251.0,31093.0,32680.0,34229.0,39409.0,40847.0,44398.0,47650.0,47998.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,29974.0,31187.0,33472.0,33572.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,392542622,OC1,1610149411577,1874,100,2500,2500,2938,4366,5742.0,7666.0,9207.0,11498.0,13009.0,14802.0,16296.0,18471.0,20551.0,22148.0,26932.0,28126.0,31355.0,32919.0,35761.0,38067.0,40468.0,41677.0,43442.0,45792.0,46548.0,49352.0,51636.0,53471.0,55977.0,57072.0,58478.0,62161.0,62623.0,NaN,NaN,...,31719.0,33134.0,34149.0,37118.0,39754.0,42180.0,43416.0,45176.0,47022.0,48670.0,50243.0,51892.0,52050.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52092,5098738412,EUW1,1613480678524,1596,100,2500,2500,3787,5583,6721.0,9945.0,11413.0,12675.0,14739.0,16711.0,18217.0,21017.0,23289.0,24894.0,27740.0,29428.0,30787.0,32817.0,35460.0,37053.0,38879.0,40345.0,42757.0,46048.0,48671.0,50786.0,54312.0,55484.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,33817.0,34911.0,36201.0,37163.0,38814.0,40027.0,42065.0,42537.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52093,5098870827,EUW1,1613482617984,1991,200,2500,2500,3516,4950,6262.0,7681.0,8632.0,10468.0,12133.0,13591.0,15175.0,16401.0,19111.0,21216.0,23719.0,26401.0,28175.0,30517.0,32611.0,34867.0,36311.0,37844.0,39814.0,40834.0,44108.0,45466.0,46978.0,48310.0,49654.0,51100.0,52615.0,54271.0,55510.0,56425.0,56535.0,...,36480.0,40856.0,42992.0,45463.0,46393.0,47806.0,49745.0,51520.0,56593.0,58177.0,60014.0,61019.0,62905.0,65996.0,66605.0,NaN,

In [ ]:
df['winTeam'] = df['winTeam'].apply(lambda x: 1 if x==100 else 0)
df['winTeam'].value_counts()

0    27183
1    24914
Name: winTeam, dtype: int64

In [ ]:
df = df[df['gameDuration']>1200].copy()
df.drop(columns=['gameId', 'platformId', 'gameCreation', 'gameDuration'], inplace=True)
df.drop(columns=[col for col in df.columns if any(f'{i}' in col for i in range(21,60))], inplace=True)
df

,winTeam,B00,B01,B02,B03,B04,B05,B06,B07,B08,B09,B10,B11,B12,B13,B14,B15,B16,B17,B18,B19,B20,R00,R01,R02,R03,R04,R05,R06,R07,R08,R09,R10,R11,R12,R13,R14,R15,R16,R17,R18,R19,R20
0,1,2500,2520,2919,4522,6395.0,8868.0,10704.0,13602.0,15652.0,17816.0,20701.0,22658.0,24849.0,28407.0,31183.0,33675.0,35888.0,37751.0,39777.0,43828.0,45970.0,2500,2500,2959,4485,6292.0,7705.0,9319.0,11187.0,12777.0,14120.0,16841.0,18125.0,20381.0,22848.0,23925.0,26680.0,29118.0,30989.0,32723.0,33916.0,36384.0
1,0,2500,2500,2959,4263,5776.0,7729.0,8929.0,10173.0,12205.0,13839.0,15782.0,19075.0,21391.0,23899.0,25540.0,26843.0,28961.0,31412.0,32550.0,34332.0,35751.0,2500,2500,2938,4475,6284.0,8600.0,10525.0,13148.0,15267.0,17201.0,19911.0,22426.0,26116.0,27919.0,30370.0,34437.0,35910.0,38867.0,41312.0,43572.0,45351.0
2,1,2500,2500,3013,5542,6525.0,8674.0,9894.0,11700.0,14722.0,16613.0,19115.0,21886.0,25497.0,28222.0,31479.0,33698.0,35321.0,36982.0,40184.0,42621.0,44408.0,2500,2500,2877,4303,6362.0,8040.0,9490.0,11490.0,12869.0,14463.0,15770.0,18928.0,20408.0,22226.0,24838.0,26411.0,29498.0,31685.0,33838.0,36152.0,38505.0
3,1,2500,2500,2900,4306,5881.0,7274.0,9769.0,11411.0,12950.0,14976.0,16618.0,18858.0,22576.0,25312.0,27010.0,29251.0,31093.0,32680.0,34229.0,39409.0,40847.0,2500,2520,2908,4143,5402.0,6618.0,7733.0,8924.0,10422.0,12107.0,14110.0,15503.0,17543.0,19058.0,21487.0,22689.0,23825.0,25991.0,27320.0,28459.0,29974.0
4,1,2500,2500,2938,4366,5742.0,7666.0,9207.0,11498.0,13009.0,14802.0,16296.0,18471.0,20551.0,22148.0,26932.0,28126.0,31355.0,32919.0,35761.0,38067.0,40468.0,2500,2500,2953,4476,5989.0,7293.0,9003.0,10465.0,12175.0,13880.0,15104.0,16596.0,18818.0,20074.0,21710.0,23133.0,25676.0,27556.0,28522.0,29811.0,31719.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52092,1,2500,2500,3787,5583,6721.0,9945.0,11413.0,12675.0,14739.0,16711.0,18217.0,21017.0,23289.0,24894.0,27740.0,29428.0,30787.0,32817.0,35460.0,37053.0,38879.0,2500,2503,3119,4473,6532.0,7624.0,9100.0,10849.0,12875.0,14449.0,16202.0,17779.0,20481.0,21716.0,23217.0,25171.0,26743.0,28215.0,30054.0,31881.0,33817.0
52093,0,2500,2500,3516,4950,6262.0,7681.0,8632.0,10468.0,12133.0,13591.0,15175.0,16401.0,19111.0,21216.0,23719.0,26401.0,28175.0,30517.0,32611.0,34867.0,36311.0,2500,2500,3279,4532,6575.0,8868.0,11134.0,12461.0,14260.0,15412.0,16963.0,18779.0,20347.0,22067.0,23774.0,26190.0,27752.0,30742.0,32134.0,34839.0,36480.0
52094,1,2500,2510,2899,5202,6277.0,8718.0,10680.0,11697.0,13305.0,14989.0,17527.0,19226.0,22663.0,24385.0,26349.0,29744.0,32001.0,33651.0,36733.0,40617.0,41908.0,2500,2520,2930,4266,6177.0,7674.0,9874.0,11858.0,13323.0,14821.0,17902.0,19736.0,21252.0,22714.0,24274.0,25518.0,27009.0,30303.0,31725.0,32835.0,34616.0
52095,0,2500,2500,2994,4270,7010.0,8039.0,9654.0,11024.0,13031.0,14126.0,17162.0,18796.0,20915.0,21957.0,24047.0,25779.0,27523.0,28904.0,30101.0,31545.0,34070.0,2500,2500,2978,4887,7103.0,9150.0,11354.0,13323.0,15518.0,17603.0,19231.0,20715.0,23691.0,26231.0,29029.0,31999.0,33787.0,36139.0,37828.0,40425.0,42330.0


In [ ]:
df = df[(df['B20']-df['R20']).between(-5000,5000)].copy()
len(df)

25097

In [ ]:
d_mean = []
d_std = []
for i in range(21):
    cols = [f'B{i:02d}', f'R{i:02d}']
    m = np.nanmean(df[cols])
    s = np.nanstd(df[cols])
    for team in cols:
        df[team] = (df[team]-m)/s if s>0 else 0
    d_mean.append(m)
    d_std.append(s)
df

In [ ]:
for i in range(21):
    cols = [f'B{i:02d}', f'R{i:02d}']
    m = np.nanmax(df[cols])
    n = np.nanmin(df[cols])
    for team in cols:
        df[team] = (df[team]-n)/(m-n) if m>n else 0
df

,winTeam,B00,B01,B02,B03,B04,B05,B06,B07,B08,B09,B10,B11,B12,B13,B14,B15,B16,B17,B18,B19,B20,R00,R01,R02,R03,R04,R05,R06,R07,R08,R09,R10,R11,R12,R13,R14,R15,R16,R17,R18,R19,R20
5,0,0,0.00000,0.116950,0.358385,0.301722,0.228180,0.222222,0.307733,0.252179,0.290249,0.301112,0.289946,0.304491,0.238517,0.212583,0.230428,0.180067,0.177440,0.322663,0.296235,0.309565,0,0.000000,0.130778,0.167959,0.203444,0.396477,0.306266,0.247238,0.267870,0.223141,0.272413,0.274749,0.265266,0.271025,0.334786,0.329463,0.295348,0.294685,0.306522,0.308732,0.349092
6,0,0,0.00000,0.099565,0.242846,0.219518,0.286888,0.315288,0.365597,0.455293,0.431083,0.410489,0.435048,0.503090,0.473260,0.489470,0.416191,0.348436,0.408460,0.399498,0.410481,0.430644,0,0.000000,0.141051,0.315325,0.264753,0.471820,0.454637,0.441741,0.472229,0.444008,0.442274,0.525371,0.571158,0.469922,0.531590,0.586157,0.538381,0.519848,0.585615,0.544357,0.556041
7,0,0,0.00000,0.094824,0.153517,0.140987,0.188454,0.403175,0.336665,0.340224,0.294489,0.341404,0.352365,0.319324,0.258980,0.254836,0.252125,0.218807,0.219848,0.292180,0.262068,0.323139,0,0.000000,0.133544,0.246590,0.218599,0.187280,0.212364,0.271436,0.279826,0.226140,0.233804,0.249764,0.222002,0.233800,0.272859,0.198198,0.212240,0.207863,0.289890,0.293392,0.322775
10,1,0,0.00000,0.109838,0.295534,0.237658,0.234051,0.177945,0.169911,0.177086,0.119326,0.122184,0.164162,0.196539,0.165300,0.305803,0.223258,0.300590,0.307809,0.332370,0.290281,0.270713,0,0.000000,0.135124,0.210752,0.301263,0.264188,0.398830,0.340742,0.268244,0.237204,0.243526,0.286426,0.328719,0.347870,0.299135,0.285941,0.221952,0.297180,0.373432,0.402650,0.386020
11,1,0,0.00000,0.089293,0.222252,0.360964,0.534834,0.567419,0.616781,0.561270,0.456002,0.479200,0.571907,0.590358,0.515130,0.551264,0.579051,0.523426,0.597939,0.666485,0.631141,0.707911,0,0.000000,0.133149,0.237497,0.257176,0.244814,0.304762,0.301289,0.304732,0.381553,0.410208,0.452992,0.476803,0.424497,0.469532,0.414922,0.427129,0.392408,0.521976,0.486537,0.497425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52086,0,0,0.02001,0.126037,0.298476,0.284271,0.422505,0.417544,0.359811,0.425405,0.408334,0.482659,0.516013,0.492707,0.561861,0.583086,0.549867,0.485790,0.485846,0.527484,0.579328,0.574609,0,0.000000,0.113394,0.215566,0.341906,0.503914,0.399833,0.587322,0.523412,0.543687,0.566234,0.521336,0.519819,0.488499,0.534099,0.507106,0.440373,0.471312,0.552568,0.559751,0.533625
52089,0,0,0.00000,0.108653,0.271998,0.311366,0.282192,0.224227,0.273672,0.229390,0.194292,0.197626,0.279643,0.372724,0.358029,0.382848,0.337584,0.345842,0.379067,0.460028,0.439820,0.414209,0,0.000000,0.168313,0.301952,0.481515,0.473190,0.593818,0.471594,0.431880,0.411023,0.421987,0.408002,0.543717,0.479428,0.490130,0.443725,0.441035,0.431725,0.539808,0.542856,0.602746
52091,1,0,0.00000,0.144212,0.306499,0.266820,0.255186,0.206683,0.218701,0.136613,0.119739,0.147611,0.216966,0.193820,0.213918,0.240114,0.184304,0.192429,0.212581,0.221344,0.182364,0.252666,0,0.249625,0.311339,0.397432,0.491848,0.445988,0.413200,0.360074,0.344956,0.315893,0.284753,0.298618,0.311084,0.254045,0.261108,0.228715,0.221125,0.241974,0.301014,0.317099,0.341031
52093,0,0,0.00000,0.361122,0.392618,0.334328,0.304892,0.204344,0.253682,0.240722,0.207217,0.208002,0.182279,0.259085,0.268631,0.315112,0.344182,0.302301,0.339100,0.385920,0.392727,0.386644,0,0.000000,0.267483,0.280824,0.406200,0.537182,0.622389,0.515781,0.505604,0.395512,0.375152,0.386451,0.360939,0.330382,0.318743,0.330796,0.278958,0.351302,0.359908,0.391225,0.395434


In [ ]:
from sklearn.model_selection import train_test_split
from random import randint

In [ ]:
cols = [col for i in range(1,21) for col in [f'B{i:02d}',f'R{i:02d}']]
random_state = randint(0,1e6)
print(f'Random state: {random_state}.')
X_train, X_test, y_train, y_test = train_test_split(df[cols], df['winTeam'], test_size=0.2, random_state=random_state)

Random state: 773991.


In [ ]:
import tensorflow as tf

In [ ]:
inputs = tf.keras.Input(shape=(40,))
x = tf.keras.layers.Reshape((20,2))(inputs)
x = tf.keras.layers.LSTM(4)(x)
outputs = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
inputs = tf.keras.Input(shape=(42,))
#x = tf.keras.layers.Dense(21, activation=tf.nn.relu)(inputs)
#x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Reshape((21,2))(inputs)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(84))(x)
x = tf.keras.layers.Dense(84, activation=tf.nn.relu)(x)
outputs = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
inputs = tf.keras.Input(shape=(42,))
#x = tf.keras.layers.BatchNormalization()(inputs)
x = tf.keras.layers.Dropout(0.3)(inputs)
for _ in range(2):
  x = tf.keras.layers.Dense(21)(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
  x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(1)(x)
outputs = tf.keras.layers.Activation('sigmoid')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data = (X_test, y_test))

Epoch 1/100
628/628 [==============================] - 7s 8ms/step - loss: 0.6986 - accuracy: 0.5155 - val_loss: 0.6672 - val_accuracy: 0.6187
Epoch 2/100
628/628 [==============================] - 5s 8ms/step - loss: 0.6634 - accuracy: 0.6192 - val_loss: 0.6409 - val_accuracy: 0.6574
Epoch 3/100
628/628 [==============================] - 5s 7ms/step - loss: 0.6398 - accuracy: 0.6553 - val_loss: 0.6262 - val_accuracy: 0.6689
Epoch 4/100
628/628 [==============================] - 5s 7ms/step - loss: 0.6234 - accuracy: 0.6726 - val_loss: 0.6130 - val_accuracy: 0.6813
Epoch 5/100
628/628 [==============================] - 5s 7ms/step - loss: 0.6146 - accuracy: 0.6797 - val_loss: 0.6050 - val_accuracy: 0.6878
Epoch 6/100
628/628 [==============================] - 5s 7ms/step - loss: 0.6087 - accuracy: 0.6857 - val_loss: 0.5993 - val_accuracy: 0.6920
Epoch 7/100
628/628 [==============================] - 5s 8ms/step - loss: 0.6039 - accuracy: 0.6838 - val_loss: 0.5962 - val_accuracy: 0.6936

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
model.get_weights()

In [ ]:
model.predict(X_test)

array([[0.47332078],
       [0.84626526],
       [0.26570255],
       ...,
       [0.50808406],
       [0.81641793],
       [0.58659136]], dtype=float32)

In [ ]:
X_test['pred'] = [a[0] for a in model.predict(X_test)]


In [ ]:
X_test[y_test!=X_test['pred'].apply(lambda x: 1 if x>0.5 else 0)][['pred', 'B20', 'R20']]

,pred,B20,R20
37676,0.265703,0.216154,0.349613
22426,0.597952,0.422011,0.363655
36984,0.553242,0.437770,0.444479
39250,0.395372,0.232382,0.279555
29154,0.463808,0.303219,0.355958
...,...,...,...
26826,0.193995,0.272169,0.387632
48660,0.285941,0.551568,0.628127
52018,0.153024,0.400947,0.647475
19403,0.252535,0.252874,0.352525


In [ ]:
X_test['win'] = y_test
X_int = X_test[((X_test['pred']-0.5)*(X_test['B20']-X_test['R20']))<0][['B20','R20','pred','win']]
X_int

,B20,R20,pred,win
36984,0.437770,0.444479,0.553242,0
26146,0.355854,0.358298,0.605342,1
40721,0.428668,0.497009,0.509556,0
51119,0.515889,0.502522,0.422947,1
27967,0.579238,0.636709,0.560840,0
...,...,...,...,...
22285,0.538253,0.547563,0.522611,1
50370,0.338274,0.379726,0.513956,0
48887,0.433037,0.408488,0.496739,1
30833,0.449316,0.442451,0.490112,0


In [ ]:
X_int['pW'] = X_int['pred'].apply(lambda x: 1 if x>0.5 else 0)
X_int[X_int['pW']!=X_int['win']]

,B20,R20,pred,win,pW
36984,0.437770,0.444479,0.553242,0,1
40721,0.428668,0.497009,0.509556,0,1
51119,0.515889,0.502522,0.422947,1,0
27967,0.579238,0.636709,0.560840,0,1
21778,0.375098,0.377490,0.536102,0,1
...,...,...,...,...,...
45822,0.506111,0.526447,0.505632,0,1
4615,0.367816,0.278411,0.489672,1,0
50370,0.338274,0.379726,0.513956,0,1
48887,0.433037,0.408488,0.496739,1,0


In [ ]:
print(model.predict(X_test))
y_pred = pd.Series(model.predict(X_test).tolist(), index=y_test.index)
y_pred = y_pred.apply(lambda x: 1 if x[0]>0.5 else 0)

[[0.03069115]
 [0.892285  ]
 [0.9262565 ]
 ...
 [0.20673528]
 [0.92696875]
 [0.5848262 ]]


In [ ]:
(y_pred==y_test).mean()

0.7234610917537747

In [ ]:
print(len(model.predict(X_test)), len(X_test))

8610 8610


In [ ]:
print(y_train.mean(),y_test.mean())

0.47926708868110807 0.4771196283391405


# Experiment 1


In [ ]:
cols = list(df.columns)
print(cols, len(cols))

['winTeam', 'B00', 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B10', 'B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B17', 'B18', 'B19', 'B20', 'R00', 'R01', 'R02', 'R03', 'R04', 'R05', 'R06', 'R07', 'R08', 'R09', 'R10', 'R11', 'R12', 'R13', 'R14', 'R15', 'R16', 'R17', 'R18', 'R19', 'R20'] 43


In [ ]:
cols[1:22], cols[22:43] = cols[22:43], cols[1:22]
df2 = df.copy()
df2.columns = cols
df2 = df2.reindex(columns=df.columns)
df2['winTeam'] = 1 - df2['winTeam']
df2

,winTeam,B00,B01,B02,B03,B04,B05,B06,B07,B08,B09,B10,B11,B12,B13,B14,B15,B16,B17,B18,B19,B20,R00,R01,R02,R03,R04,R05,R06,R07,R08,R09,R10,R11,R12,R13,R14,R15,R16,R17,R18,R19,R20
0,0,0,-0.186132,-0.054533,-0.287412,-0.001848,-0.415512,-0.364549,-0.129527,-0.344083,-0.695626,0.016397,-0.462886,-0.280052,-0.027822,-0.517831,-0.181892,-0.044107,-0.103362,-0.192641,-0.432021,-0.274644,0,0.015132,-0.262189,-0.181121,0.198546,1.408020,1.424450,2.493678,2.376115,2.384382,2.850043,2.434898,2.212924,2.727421,2.712401,2.662090,2.499802,2.254304,2.099076,2.577991,2.442738
1,1,0,-0.186132,-0.163553,-0.316140,-0.017412,0.987808,1.193236,2.000537,2.011845,1.871881,2.270100,2.286588,2.919862,2.485550,2.350569,2.971899,2.508069,2.643413,2.597770,2.500250,2.267268,0,-0.186132,-0.054533,-0.925158,-1.005760,-0.377881,-0.868311,-1.230947,-0.885285,-0.929794,-0.761020,0.144415,0.283490,0.493092,0.200938,-0.115621,-0.103101,0.044122,-0.248845,-0.305693,-0.454083
2,0,0,-0.186132,-0.480228,-0.810249,0.134342,0.109753,-0.143669,0.199596,-0.257037,-0.409792,-0.769829,0.050443,-0.264987,-0.336107,-0.111493,-0.291260,0.098683,0.139308,0.169603,0.246993,0.326605,0,-0.186132,0.225802,2.749063,0.451469,1.103837,0.378176,0.427700,1.496190,1.381880,1.685752,1.941386,2.574483,2.635728,2.844138,2.671441,2.286745,1.986181,2.231303,2.211457,1.999952
3,0,0,0.015132,-0.319294,-1.269886,-1.733402,-2.119880,-2.413179,-2.587628,-2.572281,-2.373131,-1.988443,-2.139036,-1.863550,-1.906283,-1.602883,-1.804527,-2.033015,-1.845985,-1.947977,-2.089167,-2.091713,0,-0.186132,-0.360826,-0.801631,-0.801476,-1.091301,0.216714,0.113785,-0.180398,0.017709,-0.147308,0.005695,0.944675,1.193426,0.855174,0.863408,0.698023,0.486228,0.296631,1.236057,0.990501
4,0,0,-0.186132,-0.085682,-0.313267,-0.591354,-1.061510,-0.772725,-0.913773,-0.913669,-0.895627,-1.258743,-1.440320,-1.152148,-1.402717,-1.503635,-1.624008,-1.337480,-1.300326,-1.557469,-1.678601,-1.597050,0,-0.186132,-0.163553,-0.629267,-1.071909,-0.476663,-0.509219,0.208285,-0.124575,-0.127291,-0.383690,-0.241700,-0.185199,-0.374767,0.820459,0.406013,0.796473,0.569559,0.794350,0.828528,0.883064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52092,0,0,-0.155942,0.776089,-0.321885,0.465088,-0.542517,-0.647431,-0.496667,-0.251360,-0.421459,-0.452696,-0.684071,-0.224256,-0.588882,-0.832932,-0.795411,-0.936542,-1.070557,-1.059750,-1.049997,-1.002322,0,-0.186132,4.243940,2.866845,0.832800,3.096708,2.340262,1.486758,1.512275,1.463546,1.026526,1.385866,1.342502,0.986250,1.180066,0.935372,0.583040,0.533995,0.696561,0.520603,0.432624
52093,1,0,-0.186132,1.606712,-0.152394,0.548748,1.408020,1.979879,1.254309,1.059066,0.381043,0.105958,-0.044807,-0.299023,-0.414914,-0.585035,-0.381113,-0.557398,-0.189482,-0.383996,-0.151730,-0.247430,0,-0.186132,2.837072,1.048407,-0.060215,-0.453143,-1.251944,-0.910514,-0.953408,-1.136461,-1.206622,-1.564977,-0.988665,-0.836700,-0.609513,-0.295326,-0.398450,-0.267932,-0.229027,-0.143228,-0.295338
52094,0,0,0.015132,-0.205084,-0.916540,-0.225588,-0.464119,0.352342,0.599322,0.172518,-0.111458,0.795283,0.566968,0.205933,-0.094237,-0.362505,-0.654330,-0.836589,-0.342546,-0.516873,-0.760292,-0.775826,0,-0.085500,-0.366017,1.772335,-0.031031,1.172827,1.393449,0.424442,0.155487,0.028542,0.519994,0.240944,0.993218,0.733971,0.560990,1.063849,1.039215,0.824781,1.110136,1.602895,1.291267
52095,1,0,-0.186132,0.044103,0.867425,1.576007,1.850183,2.264052,2.190625,2.249330,2.206882,1.770909,1.192808,1.566803,1.648916,1.753746,1.980673,1.710326,1.692257,1.465881,1.544590,1.410893,0,-0.186132,0.127165,-0.905049,1.395069,0.108185,0.068169,-0.306580,-0.103760,-0.690626,0.252045,-0.033940,0.017900,-0.469434,-0.463534,-0.548215,-0.643447,-0.830327,-1.044481,-1.152031,-0.930603


In [ ]:
df3 = df.append(df2, ignore_index=True)
df3

,winTeam,B00,B01,B02,B03,B04,B05,B06,B07,B08,B09,B10,B11,B12,B13,B14,B15,B16,B17,B18,B19,B20,R00,R01,R02,R03,R04,R05,R06,R07,R08,R09,R10,R11,R12,R13,R14,R15,R16,R17,R18,R19,R20
0,1,0,0.015132,-0.262189,-0.181121,0.198546,1.408020,1.424450,2.493678,2.376115,2.384382,2.850043,2.434898,2.212924,2.727421,2.712401,2.662090,2.499802,2.254304,2.099076,2.577991,2.442738,0,-0.186132,-0.054533,-0.287412,-0.001848,-0.415512,-0.364549,-0.129527,-0.344083,-0.695626,0.016397,-0.462886,-0.280052,-0.027822,-0.517831,-0.181892,-0.044107,-0.103362,-0.192641,-0.432021,-0.274644
1,0,0,-0.186132,-0.054533,-0.925158,-1.005760,-0.377881,-0.868311,-1.230947,-0.885285,-0.929794,-0.761020,0.144415,0.283490,0.493092,0.200938,-0.115621,-0.103101,0.044122,-0.248845,-0.305693,-0.454083,0,-0.186132,-0.163553,-0.316140,-0.017412,0.987808,1.193236,2.000537,2.011845,1.871881,2.270100,2.286588,2.919862,2.485550,2.350569,2.971899,2.508069,2.643413,2.597770,2.500250,2.267268
2,1,0,-0.186132,0.225802,2.749063,0.451469,1.103837,0.378176,0.427700,1.496190,1.381880,1.685752,1.941386,2.574483,2.635728,2.844138,2.671441,2.286745,1.986181,2.231303,2.211457,1.999952,0,-0.186132,-0.480228,-0.810249,0.134342,0.109753,-0.143669,0.199596,-0.257037,-0.409792,-0.769829,0.050443,-0.264987,-0.336107,-0.111493,-0.291260,0.098683,0.139308,0.169603,0.246993,0.326605
3,1,0,-0.186132,-0.360826,-0.801631,-0.801476,-1.091301,0.216714,0.113785,-0.180398,0.017709,-0.147308,0.005695,0.944675,1.193426,0.855174,0.863408,0.698023,0.486228,0.296631,1.236057,0.990501,0,0.015132,-0.319294,-1.269886,-1.733402,-2.119880,-2.413179,-2.587628,-2.572281,-2.373131,-1.988443,-2.139036,-1.863550,-1.906283,-1.602883,-1.804527,-2.033015,-1.845985,-1.947977,-2.089167,-2.091713
4,1,0,-0.186132,-0.163553,-0.629267,-1.071909,-0.476663,-0.509219,0.208285,-0.124575,-0.127291,-0.383690,-0.241700,-0.185199,-0.374767,0.820459,0.406013,0.796473,0.569559,0.794350,0.828528,0.883064,0,-0.186132,-0.085682,-0.313267,-0.591354,-1.061510,-0.772725,-0.913773,-0.913669,-0.895627,-1.258743,-1.440320,-1.152148,-1.402717,-1.503635,-1.624008,-1.337480,-1.300326,-1.557469,-1.678601,-1.597050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86091,0,0,-0.155942,0.776089,-0.321885,0.465088,-0.542517,-0.647431,-0.496667,-0.251360,-0.421459,-0.452696,-0.684071,-0.224256,-0.588882,-0.832932,-0.795411,-0.936542,-1.070557,-1.059750,-1.049997,-1.002322,0,-0.186132,4.243940,2.866845,0.832800,3.096708,2.340262,1.486758,1.512275,1.463546,1.026526,1.385866,1.342502,0.986250,1.180066,0.935372,0.583040,0.533995,0.696561,0.520603,0.432624
86092,1,0,-0.186132,1.606712,-0.152394,0.548748,1.408020,1.979879,1.254309,1.059066,0.381043,0.105958,-0.044807,-0.299023,-0.414914,-0.585035,-0.381113,-0.557398,-0.189482,-0.383996,-0.151730,-0.247430,0,-0.186132,2.837072,1.048407,-0.060215,-0.453143,-1.251944,-0.910514,-0.953408,-1.136461,-1.206622,-1.564977,-0.988665,-0.836700,-0.609513,-0.295326,-0.398450,-0.267932,-0.229027,-0.143228,-0.295338
86093,0,0,0.015132,-0.205084,-0.916540,-0.225588,-0.464119,0.352342,0.599322,0.172518,-0.111458,0.795283,0.566968,0.205933,-0.094237,-0.362505,-0.654330,-0.836589,-0.342546,-0.516873,-0.760292,-0.775826,0,-0.085500,-0.366017,1.772335,-0.031031,1.172827,1.393449,0.424442,0.155487,0.028542,0.519994,0.240944,0.993218,0.733971,0.560990,1.063849,1.039215,0.824781,1.110136,1.602895,1.291267
86094,1,0,-0.186132,0.044103,0.867425,1.576007,1.850183,2.264052,2.190625,2.249330,2.206882,1.770909,1.192808,1.566803,1.648916,1.753746,1.980673,1.710326,1.692257,1.465881,1.544590,1.410893,0,-0.186132,0.127165,-0.905049,1.395069,0.108185,0.068169,-0.306580,-0.103760,-0.690626,0.252045,-0.033940,0.017900,-0.469434,-0.463534,-0.548215,-0.643447,-0.830327,-1.044481,-1.152031,-0.930603


In [ ]:
from sklearn.model_selection import train_test_split
from random import randint

In [ ]:
random_state = randint(0,1e6)
print(f'Random state: {random_state}.')
X_train, X_test, y_train, y_test = train_test_split(df3[[f'B{i:02d}' for i in range(1,21)]], df3['winTeam'], test_size=0.2, random_state=random_state)

Random state: 544421.


In [ ]:
import tensorflow as tf

In [ ]:
inputs = tf.keras.Input(shape=(20,))
x = tf.keras.layers.Dense(40, activation=tf.nn.relu)(inputs)
x = tf.keras.layers.Dense(40, activation=tf.nn.relu)(x)
outputs = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=1, epochs=10, validation_data = (X_test, y_test))

Epoch 1/10
68876/68876 [==============================] - 81s 1ms/step - loss: 0.5261 - accuracy: 0.7367 - val_loss: 0.5119 - val_accuracy: 0.7479
Epoch 2/10
68876/68876 [==============================] - 81s 1ms/step - loss: 0.5113 - accuracy: 0.7460 - val_loss: 0.5158 - val_accuracy: 0.7481
Epoch 3/10
68876/68876 [==============================] - 80s 1ms/step - loss: 0.5126 - accuracy: 0.7466 - val_loss: 0.5059 - val_accuracy: 0.7497
Epoch 4/10
68876/68876 [==============================] - 80s 1ms/step - loss: 0.5104 - accuracy: 0.7470 - val_loss: 0.5097 - val_accuracy: 0.7482
Epoch 5/10
68876/68876 [==============================] - 80s 1ms/step - loss: 0.5115 - accuracy: 0.7469 - val_loss: 0.5097 - val_accuracy: 0.7512
Epoch 6/10
68876/68876 [==============================] - 80s 1ms/step - loss: 0.5032 - accuracy: 0.7518 - val_loss: 0.5051 - val_accuracy: 0.7507
Epoch 7/10
68876/68876 [==============================] - 79s 1ms/step - loss: 0.5094 - accuracy: 0.7479 - val_loss: 0

In [ ]:
y = model.predict(df3[[f'B{i:02d}' for i in range(1,21)]])
print(len(y))

86096


In [ ]:
y_pred = [a[0] for a in y]

In [ ]:
final_result = [1 if a-b>0 else 0 for a,b in zip(y_pred[:86096//2], y_pred[86096//2:])]
print(final_result[:10])

[1, 0, 1, 1, 1, 0, 0, 0, 1, 1]


In [ ]:
df['pred'] = final_result

In [ ]:
(df['pred']==df['winTeam']).mean()

0.8034751904850399

In [ ]:
df['prob'] = [a-b for a,b in zip(y_pred[:86096//2], y_pred[86096//2:])]
df[df['winTeam']!=df['pred']][['winTeam', 'pred', 'prob', 'B20', 'R20']]

,winTeam,pred,prob,B20,R20
9,0,1,0.663770,0.188269,-1.302237
10,1,0,-0.318274,-0.927201,-0.298739
27,1,0,-0.085732,-1.251779,-0.790000
40,0,1,0.409083,0.888734,-0.505959
60,1,0,-0.181614,-0.180814,0.115133
...,...,...,...,...,...
52065,0,1,0.174092,-0.263305,-0.240344
52074,0,1,0.332787,1.071858,0.598740
52086,0,1,0.121105,0.729138,0.505760
52091,1,0,-0.088805,-1.025567,-0.543944


In [ ]:
len(((df['B20']-df['R20'])*df['prob']).gt(0))

43048